In [1]:
import os
import sys
import time
import pickle
import random
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# 定义训练集的输入
class DataInput:
    # 创建初始化方法
    def __init__(self, data, batch_size):
        """
            data：指的是训练集数据
            batch_size：批次的大小
        """
        self.data = data
        self.batch_size = batch_size
        
        # 计算数据的批次数量
        self.epoch_size = len(self.data) // self.batch_size
        
        # 如果无法整除，则批次数加一
        if self.epoch_size * self.batch_size < len(self.data):
            # 将批次数加一
            self.epoch_size += 1
            
        # 定义一个计数器，来记录迭代的位置
        self.i = 0
        
        
    # 定义一个迭代方法
    def __iter__(self):
        # 使得 self 对象可以迭代
        return self


    # 定义 next 方法
    def __next__(self):
        # 如果迭代到了最后一个批次，则停止迭代
        if self.i == self.epoch_size:
            raise StopIteration

        # 其中，data 是一个非常大的列表，获取一个批次的数据，通过索引下标的范围来进行获取，
        # 只有最后一个批次的时候，(self.i+1)*self.batch_size 才会比 len(data) 大，这个时候需要取他们的最小值
        ts = self.data[self.i * self.batch_size: min((self.i+1)*self.batch_size, len(self.data))]

        # 更新计数器
        self.i += 1

        u, i, y, sl = [], [], [], []

        # 遍历一个批次的数据，获取一条一条记录
        for t in ts:
            # 获取用户的 id 列表
            u.append(t[0])

            # 获取候选物品的 id 列表（包括正样本和负样本）
            i.append(t[2])

            # 获取标签列表
            y.append(t[3])

            # 获取用户浏览列表的长度所构成的列表（此处获取的是一个批次的数据）
            sl.append(len(t[1]))

        # 获取一个批次数据中用户最大的浏览历史长度，因此每个批次数据中用户的最大浏览长度也是不一定的
        max_sl = max(sl)

        # 初始化一个 0 值矩阵，这个地方用 int32 还是 int64 ？？？最好和其他地方统一一下
        # 经过测试，这个地方 32 位和 64 位没有什么影响，如果是浮点型的，要注意一下
        hist_i = np.zeros(shape=[len(ts), max_sl], dtype=np.int64)

        """
            [[0, 0, 0, 0, ..., 0],
             [0, 0, 0, 0, ..., 0],
             [0, 0, 0, 0, ..., 0],
             [0, 0, 0, 0, ..., 0]]
        """

        # 用 0 值矩阵来填充用户浏览列表，因为用户浏览列表的长度是不相等的
        k = 0

        # 开始遍历数据, ts 是一个大的列表
        for t in ts:
            # t[1] 是用户的浏览历史列表，是不定长的
            for l in range(len(t[1])):
                # 开始替换数据，l 表示的是列号
                hist_i[k][l] = t[1][l] 

            # k 表示的是行号，当一行数据替换完成之后，更新到下一行
            k += 1

        # 填充后的矩阵可能是这样的
        """
            [[32, 0, 0, 0, ..., 0],
             [56, 3, 0, 0, ..., 0],
             [729, 4, 0, 0, ..., 0],
             [4, 8, 0, 0, ..., 0]]
        """

        # 返回结果，其中 self.i 可以供下一个迭代使用
        return self.i, (u, i, y, hist_i, sl)     

# 定义测试集的输入，测试集和训练集稍微有点区别
class DataInputTest:
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.epoch_size = len(self.data) // self.batch_size
        
        if self.epoch_size * self.batch_size < len(self.data):
            self.epoch_size += 1
        
        # 同样需要定义一个计数器
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        # 如果最后一个批次迭代完毕，则停止迭代
        if  self.i == self.epoch_size:
            raise StopIteration
            
        # 开始取出一个批次的数据
        ts = self.data[self.i * self.batch_size: min((self.i + 1) * self.batch_size, len(self.data))]
        
        # 取完数据之后，要更新计数器
        self.i += 1
        
        # 定义几个列表来存储数据
        u, i, j, sl = [], [], [], []
        
        for t in ts:
            # 用户的 id
            u.append(t[0])
            
            # 测试集中标签的正样本
            i.append(t[2][0])
            
            # 测试集中标签的负样本
            j.append(t[2][1])
            
            # 测试集用户浏览历史的长度
            sl.append(len(t[1]))
            
        # 获取用户浏览历史的最大长度
        max_sl = max(sl)
        
        # 构建 0 值矩阵
        hist_i = np.zeros(shape=[len(ts), max_sl], dtype=np.int64)
        
        # 开始填充
        k = 0
        
        for t in ts:
            for l in range(len(t[1])):
                hist_i[k][l] = t[1][l]
            k += 1
        
        return self.i, (u, i, j, hist_i, sl)

class Model(object):
    # 首先同样创建初始化方法
    def __init__(self, user_count, item_count, cate_count, cate_list, predict_batch_size, predict_ads_num):
        """
            user_count：总的用户数量：192403
            item_count：总的物品数量：63001
            cate_count：总的物品种类数量：801
            cate_list： 总的物品对应的种类列表
            predict_batch_size：同时预测多个物品的数据批次大小，和 b 一样大
            predict_ads_num：一次预测多个商品的数量
        """
        
        # 用户的 id 列表， [b, ]
        self.u = tf.placeholder(shape=[None, ], dtype=tf.int32)
        
        # 测试集标签的正样本商品 id，同时也可以表示训练集的候选商品 id 列表， [b, ]
        self.i = tf.placeholder(shape=[None, ], dtype=tf.int32)
        
        # 测试集标签的负样本商品 id, [b, ] 
        self.j = tf.placeholder(shape=[None, ], dtype=tf.int32)
        
        # 训练集的标签 y， [b, ]
        self.y = tf.placeholder(shape=[None, ], dtype=tf.float32)
        
        # 用户的浏览历史矩阵， [b, t]，t 表示浏览的长度
        self.hist_i = tf.placeholder(shape=[None, None], dtype=tf.int32)
        
        # 用户的浏览长度列表, [b, ]
        self.sl = tf.placeholder(shape=[None, ], dtype=tf.int32)
        
        # 学习率, []
        self.lr = tf.placeholder(shape=[], dtype=tf.float32)
        
        
        # 隐层节点的数量，也是嵌入向量的维度
        hidden_units = 128
        
        """
            优化：这个地方关于 embedding 的操作可以尝试使用 2.0 的 Embedding 工具
        """
        
        # 进行 embedding 向量的映射??? 但是通过下面的方法得到的嵌入矩阵是随机初始化的
        # 刚开始就初始化一个大的嵌入矩阵，然后通过 id 列表来获取其中对应的子嵌入矩阵
        # 所有用户的嵌入矩阵，[192403, 128]，
        user_embed_w = tf.get_variable(name="user_embed_w", shape=[user_count, hidden_units])
        
        # 所有商品的嵌入矩阵 [63001, 64]，为什么这个地方映射成 64 位，因为后面要和 categories(同样是 64 位) 拼接成 128 位
        item_embed_w = tf.get_variable(name="item_embed_w", shape=[item_count, hidden_units // 2])
        
        # 初始化 item 的偏置，初始化为 0，[63001, ]
        item_b = tf.get_variable(name="item_b", shape=[item_count, ], initializer=tf.constant_initializer(0.0))
        
        # 初始化所有物品种类的嵌入矩阵 [801, 64]
        cate_embed_w = tf.get_variable(name="cate_embed_w", shape=[cate_count, hidden_units // 2])
        
        # 将种类列表转化为张量，[63001, ]
        cate_list = tf.convert_to_tensor(cate_list, dtype=tf.int64)
        
        # tf.gather 通过索引取出对应的值，那么这个地方是通过 测试集标签中正样本商品 id 来获取对应的种类 id
        # ic 的形状为 [b, ]
        ic = tf.gather(cate_list, self.i)
        
        # 拼接一个批次的 商品嵌入矩阵和 种类嵌入矩阵
        # 拼接后的维度是 [b, 64] + [b, 64] ==> [b, 128]
        i_emb = tf.concat(values=[
            # 通过索引列表 (训练集中候选商品的 id 或者测试集标签中正样本 id) 来获取对应的子嵌入矩阵 --> [b, 64]
            tf.nn.embedding_lookup(item_embed_w, self.i), 
            tf.nn.embedding_lookup(cate_embed_w, ic)
            ], axis=1)
        
        # 获取对应的偏置  [b]
        i_b = tf.gather(item_b, self.i)
        
        # 对测试集标签中负样本商品做同样的操作
        jc = tf.gather(cate_list, self.j)
        
        # 同样进行拼接工作，拼接后的维度是 [b, 64] + [b, 64] ==> [b, 128]
        j_emb = tf.concat(values=[
            tf.nn.embedding_lookup(item_embed_w, self.i),
            tf.nn.embedding_lookup(cate_embed_w, jc)
            ], axis=1)
        # [b]
        j_b = tf.gather(item_b, self.j)
        
        
        # 获取用户浏览历史列表中的商品对应的种类
        # hc 的形状为 [b, t], t 表示浏览列表的长度
        hc = tf.gather(cate_list, self.hist_i)
        
        # 对用户浏览历史的商品嵌入矩阵和商品种类嵌入矩阵进行拼接
        # h_emb 的形状是 [b, t, 128]
        h_emb = tf.concat(values=[
            tf.nn.embedding_lookup(item_embed_w, self.hist_i),
            tf.nn.embedding_lookup(cate_embed_w, hc)
            ], axis=2)
        
        
        # 计算相关性程度，进行加权操作并进行了求和
        # 此时，hist_i 的形状是 [b, 1, h]
        hist_i = attention(i_emb, h_emb, self.sl)
        
        # 批量归一化操作  [b, 1, h]
        hist_i = tf.layers.batch_normalization(inputs=hist_i)
        
        # 重新构造形状  [b, 1, h]  --> [b, h]
        hist_i = tf.reshape(tensor=hist_i, shape=[-1, hidden_units], name='hist_bn')
        
        # 送入全连接层  [b, h]  --> [b, 128]
        # 注意：因为前面 hist_i 已经使用了名称 'hist-fcn'，所以后面hist_j 还要使用相同名称的时候，要加上 reuse=True 或者 tf.AUTO_REUSE
        hist_i = tf.layers.dense(inputs=hist_i, units=hidden_units, name="hist_fcn")
        
        # [b, 128]
        u_emb_i = hist_i
        
        # 对负样本 j 进行同样的操作
        # 首先加权求和  [b, 1, h]
        hist_j = attention(j_emb, h_emb, self.sl)
        
        # [b, 1, h]
        hist_j = tf.layers.batch_normalization(inputs=hist_j)
        
        # [b, 1, h]  --> [b, h]
        hist_j = tf.reshape(tensor=hist_j, shape=[-1, hidden_units], name='hist_bn')
        
        # [b, h] --> [b, 128]
        hist_j = tf.layers.dense(inputs=hist_j, units=hidden_units, name='hist_fcn', reuse=True)
        
        # [b, 128]
        u_emb_j = hist_j
        
        # 打印数据的形状
        print("i_emb 的形状是：{}".format(i_emb.get_shape().as_list()))
        print("j_emb 的形状是：{}".format(j_emb.get_shape().as_list()))
        print("u_emb_i 的形状是：{}".format(u_emb_i.get_shape().as_list()))
        print("u_emb_j 的形状是：{}".format(u_emb_j.get_shape().as_list()))
        
        # 进入全连接层
        # 首先拼接数据，[b, 128] + [b, 128] + [b, 128] --> [b, 128*3]
        din_i = tf.concat(values=[u_emb_i, i_emb, u_emb_i*i_emb], axis=-1)
        
        # 批量归一化
        din_i = tf.layers.batch_normalization(inputs=din_i, name='b1')
#         din_i = tf.layers.BatchNormalization(name='b1')(din_i)
        
        # 进入第一个全连接层   [b, 128*3] --> [b, 80]
        #units:输出inputs的大小，改变inputs的最后一维
        #activation：激活函数，即神经网络的非线性变化
        d_layer_1_i = tf.layers.dense(inputs=din_i, units=80, activation=tf.nn.sigmoid, name="f1")
        # 进入第二个全连接层   [b, 80] --> [b, 40]
        d_layer_2_i = tf.layers.dense(inputs=d_layer_1_i, units=40, activation=tf.nn.sigmoid, name='f2')
        # 进入第三个全连接层   [b, 40] --> [b, 1]
        d_layer_3_i = tf.layers.dense(inputs=d_layer_2_i, units=1, activation=None, name='f3')
        
        
        # 对负样本进行同样的操作  [b, 128*3]
        din_j = tf.concat(values=[u_emb_j, j_emb, u_emb_j*j_emb], axis=-1)
        
        # 批量归一化
        din_j = tf.layers.batch_normalization(inputs=din_j, name='b1', reuse=True)
#         din_j = tf.layers.BatchNormalization(name='b1', reuse=True)(din_j)
        
        # 进入第一个全连接层  [b, 128*3] --> [b, 80]
        d_layer_1_j = tf.layers.dense(inputs=din_j, units=80, activation=tf.nn.sigmoid, name="f1", reuse=True)
        
        # 进入第二个全连接层  [b, 80]  --> [b, 40]
        d_layer_2_j = tf.layers.dense(inputs=d_layer_1_j, units=40, activation=tf.nn.sigmoid, name='f2', reuse=True)
        
        # 进入第三个全连接层  [b, 40] --> [b, 1]
        d_layer_3_j = tf.layers.dense(inputs=d_layer_2_j, units=1, activation=None, name='f3', reuse=True)
        
        # 重新构造形状
        # [b, 1] --> [b]
        d_layer_3_i = tf.reshape(d_layer_3_i, [-1])
        # [b, 1] --> [b]
        d_layer_3_j = tf.reshape(d_layer_3_j, [-1])
        
        # 对所有的输出进行操作
        # 其中 i_b 和 j_b 是偏置，b 维
        # 那么，x 的形状也是 [b]
        x = i_b - j_b + d_layer_3_i - d_layer_3_j
        
        # [b]
        self.logits = i_b + d_layer_3_i
        
        # 预测选中的 item
        # 选中 item 的输出
        # 拼接所有商品和其对应种类的嵌入向量 [63001, 64] + [63001, 64] --> [63001, 128]
        item_embed_all = tf.concat(values=[
            item_embed_w, 
            tf.nn.embedding_lookup(cate_embed_w, cate_list)
            ], axis=1)
        
        # 选中前多少个商品 id 进行预测，predict_ads_num
        # 其形状是 [predict_ads_num, 128]
        item_emb_sub = item_embed_all[: predict_ads_num, :]
        
        # 扩充一个维度  [predict_ads_num, 128] --> [1, predict_ads_num, 128]
        item_emb_sub = tf.expand_dims(item_emb_sub, 0)
        
        # 开始复制数据   [1, predict_ads_num, 128] --> [predict_batch_size, predict_ads_num, 128] [b, n, h]
        item_emb_sub = tf.tile(item_emb_sub, [predict_batch_size, 1, 1])
        
        # 开始进行多物品 attention 方法
        # item_embed_sub：[predict_batch_size, predict_ads_num, 128]   [128, 100, 128]
        # h_emb: [b, t, 128]，这里 b 的大小应该也设置成了 128
        # self.sl: [b]  用户浏览历史的长度列表
        # hist_sub 的形状是 [b, n, h]
        hist_sub = attention_multi_items(item_emb_sub, h_emb, self.sl)
        
        # 批量归一化操作  [b, n, h]
        hist_sub = tf.layers.batch_normalization(inputs=hist_sub, name="hist_bn", reuse=tf.AUTO_REUSE)
#         hist_sub = tf.layers.BatchNormalization(name="hist_bn", reuse=tf.AUTO_REUSE)(hist_sub)
        
        # 重新构造 hist_sub 的形状  [b, n, h] --> [b*n, h]
        hist_sub = tf.reshape(hist_sub, [-1, hidden_units])
        
        # 送入到全连接层   [b*n, h] --> [b*n, h]
        hist_sub = tf.layers.dense(hist_sub, hidden_units, name='hist_fcn', reuse=tf.AUTO_REUSE)
        
        
        u_emb_sub = hist_sub
        
        # 重新构造 item_emb_sub 的形状  [b, n, h] --> [b*n, h]
        item_emb_sub = tf.reshape(item_emb_sub, [-1, hidden_units])
        
        # 拼接数据 3* [b*n, h] --> [b*n, 3h]
        din_sub = tf.concat([u_emb_sub, item_emb_sub, u_emb_sub*item_emb_sub], axis=-1)
        
        # 进行批量归一化操作
#         din_sub = tf.layers.batch_normalization(inputs=din_sub, name='b1', reuse=True)
        din_sub = tf.layers.BatchNormalization(name='b1', reuse=True)(din_sub)
        
        # 送入第一个全连接层  [b*n, 3h] --> [b*n, 80]
        d_layer_1_sub = tf.layers.dense(din_sub, 80, activation=tf.nn.sigmoid, name='f1', reuse=True)
        
        # 送入第二个全连接层  [b*n, 80] --> [b*n, 40]
        d_layer_2_sub = tf.layers.dense(d_layer_1_sub, 40, activation=tf.nn.sigmoid, name='f2', reuse=True)
        
        # 送入第三个全连接层  [b*n, 40] --> [b*n, 1]
        d_layer_3_sub = tf.layers.dense(d_layer_2_sub, 1, activation=None, name='f3', reuse=True)
        
        # 重新构造 d_layer_3_sub 的形状  [b*n, 1] --> [b, n]
        d_layer_3_sub = tf.reshape(d_layer_3_sub, [-1, predict_ads_num])
        
        # 加上偏置以后经过 sigmoid 函数
        self.logits_sub = tf.sigmoid(item_b[: predict_ads_num] + d_layer_3_sub)
        
        # 重构形状 [b, n] --> [b, n, 1]
        self.logits_sub = tf.reshape(self.logits_sub, [-1, predict_ads_num, 1])
        
        
        # 计算一些指标
        # x = i_b - j_b + din_layer_3_i - din_layer_3_j，其形状是 [b]，没有经过 sigmoid 函数处理
        # 也就是说 x 的值如果大于 0，经过 sigmoid 函数之后，其值为 0.5，会被预测成 1，反之被预测成 0 
        # x > 0 计算出的结果是一个布尔型值的列表，经过 float 之后，会变成 0 和 1 的列表
        # 统计预测值 1 在一个批次数据中的占比
        # self.mf_auc = tf.reduce_mean(tf.to_float(x > 0))
        # 考虑使用 tf.cast 函数来进行数据类型的转换
        """
            使用的是测试集的数据计算得出
        """
        self.mf_auc = tf.reduce_mean(tf.cast(x > 0, dtype=tf.float64))
        
        # 值为 0 或者 1 的列表 [b]，正样本计算出来的得分
        self.score_i = tf.sigmoid(i_b + d_layer_3_i)
        
        # 值同样为 0 或者 1 的列表 [b]，负样本计算出来的得分
        self.score_j = tf.sigmoid(j_b + d_layer_3_j)
        
        # 重新构造形状  [b] --> [b, 1]
        self.score_i = tf.reshape(self.score_i, [-1, 1])
        # [b] --> [b, 1]
        self.score_j = tf.reshape(self.score_j, [-1, 1])
        
        # 为了和测试集的 label 对应？？？ [b] + [b] --> [b, 2]
        self.p_and_n = tf.concat([self.score_i, self.score_j], axis=-1)
        
        # 打印一下形状看看
        print("self.p_and_n 的形状是：{}".format(self.p_and_n.get_shape().as_list()))
        
        
        # step variable
        # 设置全局步数，默认为 0，不可训练的变量，其实也就是常量
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        
        # 设置全局批次步骤
        self.global_epoch_step = tf.Variable(0, trainable=False, name="global_epoch_step")
        
        # 这个 API 的作用是给 self.global_epoch_step 重新赋值，按照后面的表达式进行赋值
        self.global_epoch_step_op = tf.assign(self.global_epoch_step, self.global_epoch_step + 1)
        
        # 计算损失值，求 b 个交叉熵的平均值
        self.loss = tf.reduce_mean(
            # 计算出来的数据的形状是 [b]
            tf.nn.sigmoid_cross_entropy_with_logits(
                # 传入预测值和真实值即可
                labels=self.y,       # [b]
                logits=self.logits   # [b]
                # self.logits = i_b + d_layer_3_i
            )
        )
        
        # 获取可训练的参数
        trainable_params = tf.trainable_variables()
        
        # 定义梯度下降优化器，并设置学习率
        self.opt = tf.train.GradientDescentOptimizer(learning_rate=self.lr)
        
        # 计算梯度，需要传入损失值和参数
        gradients = tf.gradients(self.loss, trainable_params)
        
        """
            tf.clip_by_global_norm 理解
            Gradient Clipping 的引入是为了处理 gradient explosion 或者 gradients vanishing 的问题。
            当在一次迭代中权重的更新过于迅猛的话，很容易导致 loss divergence。Gradient Clipping 的直
            观作用就是让权重的更新限制在一个合适的范围。
            
            具体的细节是
            １．在solver中先设置一个clip_gradient
            ２．在前向传播与反向传播之后，我们会得到每个权重的梯度 diff，这时不像通常那样直接使用这些梯度进行权重更新，
                而是先求所有权重梯度的平方和 sumsq_diff，如果 sumsq_diff > clip_gradient，则求缩放因子
                scale_factor = clip_gradient / sumsq_diff。这个 scale_factor 在 (0,1) 之间。如果权重梯度的平方和 
                sumsq_diff 越大，那缩放因子将越小。
            ３．最后将所有的权重梯度乘以这个缩放因子，这时得到的梯度才是最后的梯度信息。

            这样就保证了在一次迭代更新中，所有权重的梯度的平方和在一个设定范围以内，这个范围就是 clip_gradient.
            
            tf.clip_by_global_norm(t_list, clip_norm, use_norm=None, name=None) 
            
            通过权重梯度的总和的比率来截取多个张量的值。
            t_list 是梯度张量， clip_norm 是截取的比率, 这个函数返回截取过的梯度张量和一个所有张量的全局范数。        
        """
        clip_gradients, _ = tf.clip_by_global_norm(gradients, 5)
        
        # 使用优化器来更新参数
        self.train_op = self.opt.apply_gradients(zip(clip_gradients, trainable_params), global_step=self.global_step)
        
        
    # 定义训练函数  return self.i, (u, i, y, hist_i, sl) 
    def train(self, sess, uij, l):
        # sess 表示会话，uij 表示数据
        loss, _ = sess.run([self.loss, self.train_op], feed_dict={
            # 以字典的方式传入参数，对应着初始化方法中的 placeholder 的参数
            # 训练集中用户的 id
            self.u: uij[0],  
            # 训练集中候选物品的 id
            self.i: uij[1],
            # 训练集的标签
            self.y: uij[2],
            # 用户的浏览矩阵
            self.hist_i: uij[3],
            # 用户的浏览历史长度列表
            self.sl: uij[4],
            self.lr: 1,
        })
        
        return loss
    
    # 定义评估方法
    def eval(self, sess, uij):
        u_auc, score_p_and_n = sess.run([self.mf_auc, self.p_and_n], feed_dict={
            self.u: uij[0],
            self.i: uij[1],
            self.j: uij[2],
            self.hist_i: uij[3],
            self.sl: uij[4],
        })
        
        return u_auc, score_p_and_n
    
    # 定义测试方法
    def test(self, sess, uij):
        # 直接返回 self.logits_sub
        return sess.run(self.logits_sub, feed_dict={
            self.u: uij[0],
            self.i: uij[1],
            self.j: uij[2],
            self.hist_i: uij[3],
            self.sl: uij[4],
        })
    
    def save(self, sess, path):
        saver = tf.train.Saver()
        saver.restore(sess, save_path=path)
    
    def restore(self, sess, path):
        saver = tf.train.Saver()
        saver.restore(sess, save_path=path)
        
def extract_axis_1(data, ind):
    batch_range = tf.range(tf.shape(data)[0])
    indices = tf.stack([batch_range, ind], axis=1)
    res = tf.gather_nd(data, indices)
    return res

# 定义 attention 方法
def attention(queries, keys, keys_length):
    """
        queries: [b, 128]     --> i_emb
        keys: [b, t, 128]     --> h_emb
        keys_length: [b]      --> self.sl
    """

    # 获取嵌入向量的维度：h 本实验中是 128
    queries_hidden_units = queries.get_shape().as_list()[-1]

    # 对 queries 的数据进行复制 t 倍，方便进行并行化操作
    # 在第 1 个维度上复制 1 倍，在第 2 个维度上复制 t 倍
    # 此时 queries 的形状是 [b, t*128]
    queries = tf.tile(queries, [1, tf.shape(keys)[1]])

    # 重新构造 queries 的形状
    # [b, t*128] --> [b, t, 128]
    queries = tf.reshape(queries, [-1, tf.shape(keys)[1], queries_hidden_units])

    # 激活单元部分
    # 其中，queries * keys 表示对应元素相乘操作(等价于 tf.multiply() )，其形状为 [b, t, 128]
    # [b, t, 128] + [b, t, 128] + [b, t, 128] + [b, t, 128] ==> [b, t, 128*4]
    din_all = tf.concat([queries, keys, queries-keys, queries*keys], axis=-1)

    # 送入到全连接层进行训练
    # [b, t, 128*4] --> [b, t, 80]
    d_layer_1_all = tf.layers.dense(inputs=din_all, units=80, activation=tf.nn.sigmoid, name="f1_att", reuse=tf.AUTO_REUSE)

    # [b, t, 80] --> [b, t, 40]
    d_layer_2_all = tf.layers.dense(inputs=d_layer_1_all, units=40, activation=tf.nn.sigmoid, name="f2_att", reuse=tf.AUTO_REUSE)

    # [b, t, 40] --> [b, t, 1]
    d_layer_3_all = tf.layers.dense(inputs=d_layer_2_all, units=1, activation=None, name="f3_att", reuse=tf.AUTO_REUSE)
    print("d_layer_3_all 的形状是：{}".format(d_layer_3_all.get_shape().as_list()))

    # 重新构造输出数据的形状 [b, t, 1] --> [b, 1, t]
    d_layer_3_all = tf.reshape(d_layer_3_all, [-1, 1, tf.shape(keys)[1]])

    outputs = d_layer_3_all

    # 思考：这种惩罚是否真的会提高性能？？？
    # 对填充的 0 值进行惩罚，mask
    # 构造出一个和用户浏览历史矩阵相同大小的布尔值矩阵，其形状为 [b, t]
    # 作用是将原用户浏览历史矩阵中的非 0 值替换为 True，填充的 0 值替换为 False
    key_masks = tf.sequence_mask(keys_length, tf.shape(keys)[1])

    """
    构造出的布尔值矩阵如下图所示：
        [[ True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False]]
    """

    # 在第一个位置处新增一个维度  [b, t] --> [b, 1, t]
    key_masks = tf.expand_dims(input=key_masks, axis=1)

    # 构造出一个惩罚项矩阵  [b, 1, t]
    paddings = tf.ones_like(outputs) * (-2**32+1)

    # 其中 key_masks 矩阵中，True 的地方用 outputs 的值填充，False 的地方用 paddings 的值填充
    # 这里应该是对 0 值填充的地方计算出来的权重进行替换，也就是进行惩罚
    # [b, 1, t]
    outputs = tf.where(key_masks, outputs, paddings)

    # 进行放缩操作，除以嵌入向量维度的根号值，也就是根号 h，根号 128
    # [b, 1, t]
    outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)

    # 进行 softmax 归一化操作，将权重值放缩到 0-1 之间
    # [b, 1, t]
    outputs = tf.nn.softmax(outputs)

    """
        以后搭建自己模型的时候，这个地方只进行加权操作，不进行求和操作
    """

    # 进行加权操作并求和   [b, 1, t] * [b, t, h]  --> [b, 1, h]
    outputs = tf.matmul(outputs, keys)

    return outputs


# 本来一次只预测一个商品（b 次共预测 b 个商品），现在一次预测 n 个商品（b 次就是预测 b*n 个商品）
def attention_multi_items(queries, keys, keys_length):
    """
        queries: [b, n, h]  n 是商品的数量
        keys: [b, t, h]
        keys_length: [b]
    """

    # 获取嵌入向量的维度  h: 128
    queries_hidden_units = queries.get_shape().as_list()[-1]

    # 获取预测商品的数量  n: predict_ads_num
    queries_nums = queries.get_shape().as_list()[1]

    # 对 queries 的数据进行复制  [b, n, h] --> [b, n, h*t]
    queries = tf.tile(queries, [1, 1, tf.shape(keys)[1]])

    # 重新构造形状  [b, n, h*t] --> [b, n, t, h]
    queries = tf.reshape(queries, [-1, queries_nums, tf.shape(keys)[1], queries_hidden_units])

    # 最大的用户浏览长度
    max_len = tf.shape(keys)[1]

    # 对 keys 也进行数据的复制  [b, t, h] --> [b, t*n ,h]
    keys = tf.tile(keys, [1, queries_nums, 1])

    # 对 keys 重新构造形状  [b, t*n ,h] --> [b, n, t, h]
    keys = tf.reshape(keys, [-1, queries_nums, max_len, queries_hidden_units])

    # 拼接特征  [b, n, t, h] * 4 --> [b, n, t, h*4]
    din_all = tf.concat([queries, keys, queries-keys, queries*keys], axis=-1)

    # 第一层全连接层   [b, n, t, h*4] --> [b, n, t, 80]
    din_layer_1_all = tf.layers.dense(din_all, 80, activation=tf.nn.sigmoid, name='f1_att', reuse=tf.AUTO_REUSE)

    # 第二层全连接层   [b, n, t, 80] --> [b, n, t, 40]
    din_layer_2_all = tf.layers.dense(din_layer_1_all, 40, activation=tf.nn.sigmoid, name='f2_att', reuse=tf.AUTO_REUSE)

    # 第三层全连接层   [b, n, t, 40] --> [b, n, t, 1]
    din_layer_3_all = tf.layers.dense(din_layer_2_all, 1, activation=None, name='f3_att', reuse=tf.AUTO_REUSE)

    # 重新构造形状     [b, n, t, 1] --> [b, n, 1, t]
    din_layer_3_all = tf.reshape(din_layer_3_all, [-1, queries_nums, 1, max_len])

    outputs = din_layer_3_all

    # 进行惩罚操作，构造出布尔矩阵  [b, t]
    key_masks = tf.sequence_mask(keys_length, max_len)

    """
    该布尔矩阵的形状如下：
        [[ True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False]]
    """

    # 对 mask 矩阵进行数据的复制，因为要预测 n 个商品，所以要复制 n 倍数据
    # 那么，此时 masks 的形状是 [b, n*t]
    key_masks = tf.tile(key_masks, [1, queries_nums])

    # 重新构造 key_masks 的形状  [b, n*t] --> [b, n, 1, t]
    key_masks = tf.reshape(key_masks, [-1, queries_nums, 1, max_len])

    # 创建惩罚矩阵
    paddings = tf.ones_like(outputs) * (-2**32+1)

    # 开始替换数据，outputs 表示计算出来的权重矩阵，现在对填充 0 值的地方所计算出来的权重进行替换，也就是进行惩罚
    # key_masks 矩阵中值为 True 的地方用 outputs 的值代替，False 的地方用 paddings 的值代替 [b, n, 1, t]
    outputs = tf.where(key_masks, outputs, paddings)

    # 进行放缩操作 scale
    outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)

    # 进行 softmax 操作  [b, n, 1, t]
    outputs = tf.nn.softmax(outputs)

    # 重新构造 outputs 的形状  [b, n, 1, t] --> [b*n, 1, t]
    outputs = tf.reshape(outputs, [-1, 1, max_len])

    # 重新构造 keys 的形状，[b, n, t, h] --> [b*n, t, h]
    keys = tf.reshape(keys, [-1, max_len, queries_hidden_units])

    # 进行加权操作并求和  [b*n, 1, t]*[b*n, t, h]  --> [b*n, 1, h]
    outputs = tf.matmul(outputs, keys)

    # 重新构造 outputs 的形状   [b*n, 1, h] --> [b, n, h]
    outputs = tf.reshape(outputs, [-1, queries_nums, queries_hidden_units])

    # [b, n, h]
    print("预测多个项目的输出是：{}".format(outputs.get_shape().as_list()))

    return outputs

# 设置随机数的种子
random.seed(2021)
np.random.seed(2021)
tf.set_random_seed(2021)

# 设置超参数
train_batch_size = 32
test_batch_size = 512
predict_batch_size = 32
predict_users_num = 1000
predict_ads_num = 100

with open('./raw_data/dataset.pkl', 'rb') as f:
    # 按顺序加载数据
    train_set = pickle.load(f)
    test_set = pickle.load(f)
    cate_list = pickle.load(f)
    user_count, item_count, cate_count = pickle.load(f)
    #192403,63001,801
    
# 记录最佳的 auc 值，初始化为 0
best_auc = 0.0

# 定义计算 auc 的方法
def calc_auc(raw_arr):
    # 对 raw_arr 按负样本和正样本的预测得分，从小到大进行排序
    arr = sorted(raw_arr, key=lambda d: d[2])
    
    auc = 0.0
    
    """
    混淆矩阵的四个值介绍：
        TN：真阴性，表示实际是负样本且预测值也为负样本的样本数
        FP：假阳性，表示实际值是负样本预测值却为正样本的样本数
        FN：假阴性，表示实际值是正样本预测值却为负样本的样本数
        TP：真阴性，表示实际值是正样本预测值也为正样本的样本数
    """
    
    fp1, tp1, fp2, tp2 = 0.0, 0.0, 0.0, 0.0
    
    for record in arr:
        fp2 += record[0]   # 未点击
        tp2 += record[1]   # 点击
        auc += (fp2 - fp1) * (tp2 + tp1)
        fp1, tp1 = fp2, tp2
        
    # 如果所有的都是未点击 或者 点击，丢弃
    # 定义阈值
    threshold = len(arr) - 1e-3
    
    if tp2 > threshold or fp2 > threshold:
        return -0.5
    
    if tp2 * fp2 > 0.0:   # 正常的 auc
        return (1.0 - auc / (2.0 * tp2 * fp2))
    else:
        return None
    
def _auc_arr(score):
    # 正样本的预测得分 [0-1]
    score_p = score[:, 0]
    # 负样本的预测得分 [0-1]
    score_n = score[:, 1]
    
    # 定义一个分数数组
    score_arr = []
    
    for s in score_p.tolist():
        score_arr.append([0, 1, s])
    for s in score_n.tolist():
        score_arr.append([1, 0, s])
    
    # len(score_arr) = 2b
    return score_arr

def _eval(sess, model):
    auc_sum = 0.0
    score_arr = []
    
    # 输入测试集的数据，批次为 test_batch_size = 512
    # uij 是指一批次的数据，DataInputTest() 返回的是 return self.i, (u, i, j, hist_i, sl)
    for _, uij in DataInputTest(test_set, test_batch_size):

        # eval() 方法返回的是 u_auc, score_p_and_n，其形状分别是 [b] 和 [b, 2]
        auc_, score_ = model.eval(sess, uij)

        # 拼接列表(所有批次的数据集计算出来的得分) len(score_arr) = len(test_set)
        score_arr += _auc_arr(score_)

        # 得到的是整个测试集中 1 的个数
        auc_sum += auc_ * len(uij[0])

    # 得到的是整个测试集中 1 的占比
    test_gauc = auc_sum / len(test_set)

    Auc = calc_auc(score_arr)
    
    global best_auc
    
    if best_auc < test_gauc:
        best_auc = test_gauc
#         model.save(sess, "save_path/ckpt")
    
    return test_gauc, Auc

def _test(sess, model):
    auc_sum = 0.0
    score_arr = []
    predicted_users_num = 0
    print("test sub items")
    
    # uij 是指一批次的数据，DataInputTest() 返回的是 return self.i, (u, i, j, hist_i, sl)
    for _, uij in DataInputTest(test_set, predict_batch_size):
        
        # 只预测前 predict_users_num 条数据
        if predicted_users_num > predict_users_num:
            break
        
        # 直接返回 self.logits_sub 形状是 [b, n, 1]
        score_ = model.test(sess, uij)
        
        score_arr.append(score_)
        
        predicted_users_num += predict_batch_size
        
    # 这个地方返回的是个啥？？？
    return score_[0]

gpu_options = tf.GPUOptions(allow_growth=True)#动态申请显存

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    # 模型实例化
    model = Model(user_count=user_count, 
                  item_count=item_count, 
                  cate_count=cate_count, 
                  cate_list=cate_list, 
                  predict_batch_size=predict_batch_size,
                  predict_ads_num=predict_ads_num)
    
    # 初始化变量
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    print("test_gauc: %.4f, test_auc: %.4f" % _eval(sess, model))
    
    """
    缓冲区的刷新方式：
        flush()刷新缓存区
        缓冲区满时，自动刷新
        文件关闭或者是程序结束自动刷新。
    """
    sys.stdout.flush()
    
    # 学习率设置为 1.0
    lr = 1.0
    
    start_time = time.time()
    
    # 开始进行迭代训练
    for _ in range(50):
        # 打乱训练集
        random.shuffle(train_set)
        
        epoch_size = round(len(train_set) / train_batch_size)
        
        loss_sum = 0.0
        
        # uij 表示一个批次的数据
        for _, uij in DataInput(train_set, train_batch_size):
            loss = model.train(sess, uij, lr)
            loss_sum += loss
            
            if model.global_step.eval() % 1000 == 0:
                test_gauc, Auc = _eval(sess, model)
                print('Epoch %d Global_step %d\tTrain_loss: %.4f\tEval_GAUC: %.4f\tEval_AUC: %.4f' %
                      (model.global_epoch_step.eval(), model.global_step.eval(),
                       loss_sum / 1000, test_gauc, Auc))
                sys.stdout.flush()
                loss_sum = 0.0

            if model.global_step.eval() % 336000 == 0:
                lr = 0.1

        print('Epoch %d DONE\tCost time: %.2f' %
              (model.global_epoch_step.eval(), time.time()-start_time))
        sys.stdout.flush()
        model.global_epoch_step_op.eval()
    
    # 打印出最好的结果
    print('best test_gauc:', best_auc)
    sys.stdout.flush()

D:\LenovoSoftstore\Install\Anac3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
D:\LenovoSoftstore\Install\Anac3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
D:\LenovoSoftstore\Install\Anac3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\normalization.py:308: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  '`tf

d_layer_3_all 的形状是：[None, None, 1]
d_layer_3_all 的形状是：[None, None, 1]
i_emb 的形状是：[None, 128]
j_emb 的形状是：[None, 128]
u_emb_i 的形状是：[None, 128]
u_emb_j 的形状是：[None, 128]
预测多个项目的输出是：[None, 100, 128]


TypeError: ('Keyword argument not understood:', 'reuse')